使用 PyTorch 实现一个简单的卷积神经网络 (CNN) 来进行手写数字识别是一个很好的练习。下面是一个完整的示例，包括数据加载、模型定义、训练和评估过程。我们将使用 PyTorch 的内置功能来处理 MNIST 数据集。

### 1. 导入必要的库

```python
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
```

### 2. 数据预处理和加载

```python
# 定义转换操作，将数据转换为tensor并进行归一化
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# 加载训练数据和测试数据
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# 创建数据加载器
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
```

### 3. 定义CNN模型

```python
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv_layer = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(64 * 7 * 7, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.conv_layer(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc_layer(x)
        return x
```

### 4. 训练模型

```python
# 实例化模型、损失函数和优化器
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# GPU支持
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 训练过程
num_epochs = 5
for epoch in range(num_epochs):
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        # 前向传播
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
```

### 5. 评估模型

```python
# 测试模型
model.eval()  # 设置为评估模式
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy of the model on the 10000 test images: {100 * correct / total:.2f}%')
```

这个例子展示了如何使用 PyTorch 构建和训练一个简单的 CNN 来识别 MNIST 数据集中的手写数字。你可以根据需要调整模型结构和参数以探索不同配置的性能。


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# 数据预处理和加载
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 定义CNN模型
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv_layer = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(64 * 7 * 7, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.conv_layer(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc_layer(x)
        return x

# 实例化模型、损失函数和优化器
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# GPU支持
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 训练模型
num_epochs = 5
for epoch in range(num_epochs):
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        # 前向传播
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# 评估模型
model.eval()  # 设置为评估模式
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy of the model on the 10000 test images: {100 * correct / total:.2f}%')

100%|██████████| 9912422/9912422 [00:03<00:00, 2862723.10it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 31710914.61it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:01<00:00, 1292306.64it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 18160656.59it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Epoch [1/5], Loss: 0.0466
Epoch [2/5], Loss: 0.0597
Epoch [3/5], Loss: 0.0988
Epoch [4/5], Loss: 0.0269
Epoch [5/5], Loss: 0.0002
Accuracy of the model on the 10000 test images: 98.95%


## 模型参数统计

In [2]:
def count_parameters_detailed(model):
    total_params = 0
    for name, parameter in model.named_parameters():
        if parameter.requires_grad:
            num_params = parameter.numel()
            total_params += num_params
            print(f"{name}: {num_params}")
    print(f'Total trainable parameters: {total_params * 1e-3} k')

# 实例化模型
model = CNN()

# 统计并打印模型的可训练参数总量
count_parameters_detailed(model)


conv_layer.0.weight: 288
conv_layer.0.bias: 32
conv_layer.3.weight: 18432
conv_layer.3.bias: 64
fc_layer.0.weight: 401408
fc_layer.0.bias: 128
fc_layer.2.weight: 1280
fc_layer.2.bias: 10
Total trainable parameters: 421.642 k


## 数据流

在提供的 CNN 模型中，数据流通过几个主要的处理步骤，从输入层到输出层。这里是一个详细的步骤描述，展示了数据如何在模型中流动：

### 1. 输入层
- **输入数据**：模型接收的输入是 MNIST 数据集的图像，这些图像是灰度图（单通道），每个图像的大小为 28x28 像素。
- **数据预处理**：输入数据首先通过一个预处理步骤，包括转换为张量，并标准化。标准化使用均值 0.5 和标准差 0.5 来调整像素值，使其范围在 [-1, 1] 之间。

### 2. 第一层卷积层
- **卷积操作**：输入图像通过一个包含 32 个过滤器的卷积层，每个过滤器的大小为 3x3，使用 padding=1 来保持图像尺寸。这个步骤帮助提取图像的基本特征。
- **ReLU激活函数**：卷积后的特征图通过 ReLU 激活函数，增加非线性，帮助网络学习复杂的模式。
- **最大池化**：接着是一个 2x2 的最大池化层，步长为 2。这一步骤减少数据的空间尺寸（从 28x28 到 14x14），减少参数数量和计算量，同时保持重要特征。

### 3. 第二层卷积层
- **卷积操作**：经过第一层处理后的特征图再次通过一个卷积层，这次是 64 个过滤器，过滤器大小仍为 3x3，使用 padding=1。
- **ReLU激活函数**：同样，卷积后的特征图通过 ReLU 激活函数。
- **最大池化**：再次应用 2x2 最大池化，步长为 2，将特征图尺寸从 14x14 减少到 7x7。

### 4. 全连接层
- **展平操作**：经过两次卷积和池化后，特征图需要被展平（flatten），从而可以被全连接层处理。展平后的向量长度为 64（过滤器数量）乘以 7（宽）乘以 7（高）= 3136。
- **第一个全连接层**：展平的特征通过一个全连接层，该层有 128 个神经元。
- **ReLU激活函数**：全连接层后接一个 ReLU 激活函数。
- **第二个全连接层**：最后，数据通过另一个全连接层，这层有 10 个输出神经元，对应于 10 个类别的数字（0到9）。

### 5. 输出层
- **输出**：最终输出是一个 10 维向量，每个维度代表一个类别的预测概率。通常，这些概率会通过 softmax 函数转换，用于分类任务。

这个数据流程描述了如何从输入图像到分类输出的过程，展示了每个步骤如何对数据进行转换和处理。


In [3]:
import torch
import torch.nn as nn
from torchviz import make_dot


# 定义CNN模型
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv_layer = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(64 * 7 * 7, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.conv_layer(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc_layer(x)
        return x

# 实例化模型、损失函数和优化器
model = CNN()

## 模型可视化 （not good）
x = torch.randn(1, 1, 28, 28)
y = model(x)
dot = make_dot(y, params=dict(list(model.named_parameters())))
dot.render('model_graph', format='png')

# 设置导出路径和文件名
output_onnx = './model.onnx'

# 导出模型
torch.onnx.export(model, x, output_onnx, export_params=True, opset_version=10,
                  do_constant_folding=True, input_names=['input'], output_names=['output'],
                  dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}})

print(f"Model has been converted to ONNX and saved to {output_onnx}")

Model has been converted to ONNX and saved to ./model.onnx


![](images/20240420094722.png)